# Combining dependency management with conda and Docker
  We can combine an environment.yml file with a Dockerfile and an install.R
  script to cover many container use cases for Bioinformatics.

### **Overview**

The [`nf-core`](https://github.com/nf-core) community has established a highly practical convention for creating  docker containers by re-using an **environment.yml** file that can be also used for installing dependencies in an interactive session.

There are 3 main ingredients to this Dockerfile formula:

1. The **environment.yml** file
2. The template **Dockerfile**
3. An **install.R** file _**\(optional\)**_ 

 The latter is only optional and can be possibly proven useful for R libraries that are _**not**_  available via **conda**.

### Templates for each of the 3 files

1. **environment.yml**

This file has no difference when used within a Dockerfile installation or when used natively for installing dependencies via conda. We will utilise this to create a concise Dockerfile.

```yaml
name: my-new-env
channels:
  - conda-forge
  - bioconda
  - r
dependencies:
  - python
  - r-base
  - r-devtools=2.2
```

**2. install.R**   
_**Optional**_ but frequently useful, when dependencies are not available via conda. The file requires 2 lines that will prevent issues that come with non-interactive R installation.

```r
Sys.setenv(TAR = "/bin/tar")
options(repos = "https://cloud.r-project.org/")
install.packages("remotes")
install.packages("BiocManager")
```

**3. Dockerfile**  
  
The Dockerfile is the most integral file in the process, it constitutes the main installation blueprint in which the auxillary files **`environment.yml`** ``and **`install.R`** will be included in.

The only part of the following **Dockerfile** that may be useful to update is the name of the environment:


```Dockerfile
# Snippet to update (See in the full template below the line 5)
ARG ENV_NAME="my-test-env"
```

```Dockerfile
# Full contents of Dockerfile

FROM continuumio/miniconda3:4.8.2
LABEL description="Base docker image with conda and util libraries"
ARG ENV_NAME="my-test-env"

# Install mamba for faster installation in the subsequent step
# Install r-base for being able to run the install.R script
RUN conda install -c conda-forge mamba r-base -y

# Install the conda environment
COPY environment.yml /
RUN mamba env create --quiet --name ${ENV_NAME} --file /environment.yml && conda clean -a

# Install R packages that are possibly not available via conda
COPY bin/install.R /
RUN Rscript /install.R

# Add conda installation dir to PATH (instead of doing 'conda activate')
ENV PATH /opt/conda/envs/${ENV_NAME}/bin:$PATH

# Dump the details of the installed packages to a file for posterity
RUN mamba env export --name ${ENV_NAME} > ${ENV_NAME}_exported.yml

# Copy additional scripts from bin and add to PATH
RUN mkdir /opt/bin
COPY bin/* /opt/bin/
RUN chmod +x /opt/bin/*
ENV PATH="$PATH:/opt/bin/"
```

###  **Combining all the files to build the docker image**🐳 

To build the docker container and create the docker image we will need the auxillary files to be in the same directory or a subdirectory of the root repo of the Dockerfile.  
  
It is typical that all files reside in the same directory. An also common practice is to create a helper folder named `bin` to hold all of the scripts and executables. To proceed withh the installation commands, make sure that your files in your directory look similar to this if you want to use the template without updating any paths:

```bash
➜ tree
.
├── Dockerfile
├── bin
│   └── install.R
├── environment.yml
```

The command to build the docker container is the following, when the Dockerfile resides in the current directory:  

```bash
docker build -t lifebitai/test-docker-image:v0.1 .
```

It is advised to you the tag, the mark after the colon to label the version of the docker image that will be created.  
  

### Pushing the docker image to the Docker Hub registry

After the successful build process we are able to store in a registry the created docker image.  
To do so, make sure you have used the login command from the terminal and have followed the command prompt to authenticate:

```bash
docker login
```

After successfully logging in you will be able to run the final command to push the docker image by typing:  


```bash
docker push lifebitai/test-docker-image:v0.1
```

### Testing or running the docker image

To use the created docker image, a simple yet very practical way is to mount as a volume the current working directory. This will ensure access to the files available in the folder but also allow the generated results of the analysis to be retained after exiting the docker container.  
  
To run the docker container with the current working directory mounted, type the following command:  


```bash
docker run -v $PWD:$PWD -w $PWD -it  lifebitai/test-docker-image:v0.1
```